In [2]:
import markdown
import urllib2
import re
import pandas as pd

In [3]:
currURL = 'https://raw.githubusercontent.com/sindresorhus/awesome/master/readme.md'
# Assign the open file to a variable
webFile = urllib2.urlopen(currURL)

# Read the file contents to a variable
file_contents = webFile.read()


In [4]:
from markdown.treeprocessors import Treeprocessor

class CodeBlockTweaker(Treeprocessor):
    def __init__(self, md):
        self.cur_level = []
        self.last_a = False
        self.output = []
        super(CodeBlockTweaker, self).__init__(md)
    def run(self, root):
        return self.setClass(root)
    def setClass(self, element):
        for child in element:
            if child.tag == "a":
                #print(self.cur_level, child.text, child.get("href"))
                #Out put only github repos
                
                href = child.get("href")                    
                is_repo = re.match('^https://github.com/\w+', href)
                if is_repo is not None:                    
                    self.output.append((self.cur_level + [child.text], href))
                self.last_a = child.text
            # run recursively on children
            if child.tag in ["h2", "h3"]:
                if  self.cur_level:
                    self.cur_level.pop()
                self.cur_level.append(child.text)
                self.last_a = False
            elif child.tag == "ul" and self.last_a:
                self.cur_level.append(self.last_a)
                
            child = self.setClass(child)
            
            if child.tag == "ul" and self.last_a and self.cur_level:                
                self.cur_level.pop()
        return element

from markdown.extensions import Extension

class CodeBlockExtension(Extension):
    def extendMarkdown(self, md, md_globals):
        self.tweaker = CodeBlockTweaker(md)
        md.treeprocessors.add('codeblocktweaker', self.tweaker, '_end')

ext = CodeBlockExtension()
        
html = markdown.markdown(text=unicode(file_contents, 'utf-8'), extensions=[ext])

index_repos = pd.DataFrame(ext.tweaker.output, columns=["categories", "repo_url"])
    

In [5]:
#input_file = codecs.open("some_file.txt", mode="r", encoding="utf-8")
#text = input_file.read()
#html = markdown.markdown(unicode(file_contents, 'utf-8'))

In [6]:
index_repos.describe()

,categories,repo_url
count,327,327
unique,327,327
top,"[Books, R Books]",https://github.com/wbkd/awesome-d3
freq,1,1


In [7]:
index_repos

,categories,repo_url
0,"[Platforms, Node.js]",https://github.com/sindresorhus/awesome-nodejs
1,"[Platforms, Frontend Development]",https://github.com/dypsilon/frontend-dev-bookm...
2,"[Platforms, iOS]",https://github.com/vsouza/awesome-ios
3,"[Platforms, Android]",https://github.com/JStumpp/awesome-android
4,"[Platforms, IoT & Hybrid Apps]",https://github.com/weblancaster/awesome-IoT-hy...
5,"[Platforms, Electron]",https://github.com/sindresorhus/awesome-electron
6,"[Platforms, Cordova]",https://github.com/busterc/awesome-cordova
7,"[Platforms, React Native]",https://github.com/jondot/awesome-react-native
8,"[Platforms, Xamarin]",https://github.com/benoitjadinon/awesome-xamarin
9,"[Platforms, Linux]",https://github.com/aleksandar-todorovic/awesom...


In [8]:
URL2 = 'https://raw.githubusercontent.com/bayandin/awesome-awesomeness/master/README.md'

ext = CodeBlockExtension()
        
html = markdown.markdown(text=unicode(urllib2.urlopen(URL2).read(), 'utf-8'), extensions=[ext])

index_repos2 = pd.DataFrame(ext.tweaker.output, columns=["categories", "repo_url"])

In [13]:
index_repos2['categories'][0]

[u'Awesome-Awesomeness.ZEEF.com',
 u'Awesome-Awesomeness.ZEEF.com',
 u'AutoHotkey']